# Develop data & tools to add interactive clustering model to Dash app 

In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import numpy as np
import pandas as pd
import pickle
from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt
# import seaborn as sns
from src.data.process_SDG import load_2017_sdg_data

In [24]:
N_CLUSTERS = 4
from src.visualization.utils import palette
colorscale = [[k/(N_CLUSTERS-1), palette[k]] for k in range(0, N_CLUSTERS)]  #choropleth colorscale seems to need 0-1 range
colorscale


[[0.0, '#ff0000'],
 [0.3333333333333333, '#ff8000'],
 [0.6666666666666666, '#fff100'],
 [1.0, '#40ff00']]

In [11]:
with open('../data/metadata/indicator_dictionary.pickle', 'rb') as handle:
    indicator_dict = pickle.load(handle)

df = load_2017_sdg_data()
df.head()

,location_id,location_name,year_id,estimate_type,indicator_id,indicator_short,ihme_indicator_description,indicator_outline,indicator_unit,target_description,goal_description,scaled_value,scaled_lower,scaled_upper
0,6,China,2017,past,1000,HIV Incid,Indicator 3.3.1: Age-standardised rate of new ...,3.3.1,Scaled/Normalized value ranging from 0-100,"By 2030, end the epidemics of AIDS, tuberculos...",Ensure healthy lives and promote well-being fo...,74.5,67.6,82.8
1,7,North Korea,2017,past,1000,HIV Incid,Indicator 3.3.1: Age-standardised rate of new ...,3.3.1,Scaled/Normalized value ranging from 0-100,"By 2030, end the epidemics of AIDS, tuberculos...",Ensure healthy lives and promote well-being fo...,76.9,41.1,100.0
2,8,Taiwan,2017,past,1000,HIV Incid,Indicator 3.3.1: Age-standardised rate of new ...,3.3.1,Scaled/Normalized value ranging from 0-100,"By 2030, end the epidemics of AIDS, tuberculos...",Ensure healthy lives and promote well-being fo...,84.5,77.7,90.7
3,10,Cambodia,2017,past,1000,HIV Incid,Indicator 3.3.1: Age-standardised rate of new ...,3.3.1,Scaled/Normalized value ranging from 0-100,"By 2030, end the epidemics of AIDS, tuberculos...",Ensure healthy lives and promote well-being fo...,63.3,49.3,81.1
4,11,Indonesia,2017,past,1000,HIV Incid,Indicator 3.3.1: Age-standardised rate of new ...,3.3.1,Scaled/Normalized value ranging from 0-100,"By 2030, end the epidemics of AIDS, tuberculos...",Ensure healthy lives and promote well-being fo...,59.8,55.3,63.4


In [20]:
N_CLUSTERS = 7
INDICATORS_TO_INCLUDE = list(indicator_dict.keys())
# INDICATORS_TO_INCLUDE = ['Adol Birth Rate', 'Air Poll Mort']

In [21]:
data = df.pivot(index='location_name', columns='indicator_short', values='scaled_value')
data = data[INDICATORS_TO_INCLUDE]
data.head()

indicator_short,Adol Birth Rate,Air Poll Mort,Alcohol Use,Cert Death Reg,Child Overweight,Child Sex Abuse,Child Stunting,Child Wasting,Conflict Mort,Disaster Mort,...,Sexual Violence,Skilled Birth Attend,Smoking Prev,Suicide Mort,TB Incid,UHC Index,Under-5 Mort,Vaccine Cov,WaSH Mort,Water
location_name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,14.8,5.7,96.6,0.0,96.6,68.2,14.2,74.6,13.1,27.6,...,88.3,49.7,82.5,51.6,34.7,26.5,27.7,46.4,31.2,4.3
Albania,59.6,55.4,53.3,77.6,22.4,75.0,63.6,62.9,100.0,69.3,...,98.9,99.4,24.9,76.7,79.3,78.5,62.2,87.8,85.5,79.1
Algeria,77.8,53.2,94.5,62.8,35.5,67.9,78.7,78.7,37.3,100.0,...,87.2,96.2,59.0,85.7,61.3,66.4,48.2,82.6,67.9,86.1
American Samoa,42.2,42.0,84.9,77.3,37.4,6.6,83.3,79.6,100.0,100.0,...,37.8,99.2,28.9,64.6,76.7,53.3,67.5,24.7,53.8,77.0
Andorra,95.9,90.7,19.6,0.0,19.6,60.2,98.9,99.2,100.0,100.0,...,88.8,99.8,34.9,57.9,92.5,99.0,100.0,85.0,95.4,100.0


In [22]:
kmean = KMeans(n_clusters=N_CLUSTERS, random_state=0)
kmean.fit(data)
data['cluster'] = kmean.labels_
data.head()

indicator_short,Adol Birth Rate,Air Poll Mort,Alcohol Use,Cert Death Reg,Child Overweight,Child Sex Abuse,Child Stunting,Child Wasting,Conflict Mort,Disaster Mort,...,Skilled Birth Attend,Smoking Prev,Suicide Mort,TB Incid,UHC Index,Under-5 Mort,Vaccine Cov,WaSH Mort,Water,cluster
location_name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,14.8,5.7,96.6,0.0,96.6,68.2,14.2,74.6,13.1,27.6,...,49.7,82.5,51.6,34.7,26.5,27.7,46.4,31.2,4.3,2
Albania,59.6,55.4,53.3,77.6,22.4,75.0,63.6,62.9,100.0,69.3,...,99.4,24.9,76.7,79.3,78.5,62.2,87.8,85.5,79.1,5
Algeria,77.8,53.2,94.5,62.8,35.5,67.9,78.7,78.7,37.3,100.0,...,96.2,59.0,85.7,61.3,66.4,48.2,82.6,67.9,86.1,6
American Samoa,42.2,42.0,84.9,77.3,37.4,6.6,83.3,79.6,100.0,100.0,...,99.2,28.9,64.6,76.7,53.3,67.5,24.7,53.8,77.0,5
Andorra,95.9,90.7,19.6,0.0,19.6,60.2,98.9,99.2,100.0,100.0,...,99.8,34.9,57.9,92.5,99.0,100.0,85.0,95.4,100.0,1


# To share data between callbacks, it needs to be JSON format
https://dash.plot.ly/sharing-data-between-callbacks

In [37]:
data_j = data.to_json()

In [38]:
data_j

'{"Adol Birth Rate":{"Afghanistan":14.8,"Albania":59.6,"Algeria":77.8,"American Samoa":42.2,"Andorra":95.9,"Angola":8.7,"Antigua and Barbuda":31.8,"Argentina":28.2,"Armenia":52.4,"Australia":69.2,"Austria":79.9,"Azerbaijan":36.6,"Bahrain":65.7,"Bangladesh":23.3,"Barbados":38.5,"Belarus":59.5,"Belgium":84.7,"Belize":28.1,"Benin":18.5,"Bermuda":80.5,"Bhutan":42.4,"Bolivia":22.4,"Bosnia and Herzegovina":77.0,"Botswana":35.4,"Brazil":31.7,"Brunei":70.9,"Bulgaria":39.3,"Burkina Faso":14.1,"Burundi":31.1,"Cambodia":36.8,"Cameroon":15.8,"Canada":70.2,"Cape Verde":41.7,"Central African Republic":17.3,"Chad":0.1,"Chile":37.9,"China":80.0,"Colombia":25.9,"Comoros":36.1,"Congo":23.6,"Costa Rica":30.5,"Cote d\'Ivoire":13.6,"Croatia":77.6,"Cuba":34.8,"Cyprus":99.9,"Czech Republic":71.0,"Democratic Republic of the Congo":17.5,"Denmark":98.2,"Djibouti":33.0,"Dominica":35.5,"Dominican Republic":16.6,"Ecuador":27.7,"Egypt":27.3,"El Salvador":26.2,"Equatorial Guinea":11.8,"Eritrea":33.5,"Estonia":67.8,"

In [43]:
df_c = pd.read_json(data_j)
df_c.head()

,Adol Birth Rate,Air Poll Mort,Alcohol Use,Cert Death Reg,Child Overweight,Child Sex Abuse,Child Stunting,Child Wasting,Conflict Mort,Disaster Mort,...,Skilled Birth Attend,Smoking Prev,Suicide Mort,TB Incid,UHC Index,Under-5 Mort,Vaccine Cov,WaSH Mort,Water,cluster
Afghanistan,14.8,5.7,96.6,0.0,96.6,68.2,14.2,74.6,13.1,27.6,...,49.7,82.5,51.6,34.7,26.5,27.7,46.4,31.2,4.3,2
Albania,59.6,55.4,53.3,77.6,22.4,75.0,63.6,62.9,100.0,69.3,...,99.4,24.9,76.7,79.3,78.5,62.2,87.8,85.5,79.1,5
Algeria,77.8,53.2,94.5,62.8,35.5,67.9,78.7,78.7,37.3,100.0,...,96.2,59.0,85.7,61.3,66.4,48.2,82.6,67.9,86.1,6
American Samoa,42.2,42.0,84.9,77.3,37.4,6.6,83.3,79.6,100.0,100.0,...,99.2,28.9,64.6,76.7,53.3,67.5,24.7,53.8,77.0,5
Andorra,95.9,90.7,19.6,0.0,19.6,60.2,98.9,99.2,100.0,100.0,...,99.8,34.9,57.9,92.5,99.0,100.0,85.0,95.4,100.0,1


In [48]:
df_cluster = df_c.groupby('cluster').mean().reset_index().melt(id_vars='cluster')
df_cluster.head()

,cluster,variable,value
0,0,Adol Birth Rate,38.472727
1,1,Adol Birth Rate,82.572973
2,2,Adol Birth Rate,14.045833
3,3,Adol Birth Rate,29.554545
4,4,Adol Birth Rate,23.604545


In [53]:
df_cluster = df_c.groupby('cluster').mean()
df_cluster.head()

,Adol Birth Rate,Air Poll Mort,Alcohol Use,Cert Death Reg,Child Overweight,Child Sex Abuse,Child Stunting,Child Wasting,Conflict Mort,Disaster Mort,...,Sexual Violence,Skilled Birth Attend,Smoking Prev,Suicide Mort,TB Incid,UHC Index,Under-5 Mort,Vaccine Cov,WaSH Mort,Water
cluster,,,,,,,,,,,,,,,,,,,,,
0,38.472727,22.763636,78.054545,12.627273,83.350000,42.245455,45.640909,55.695455,82.950000,82.368182,...,60.045455,75.859091,49.513636,53.463636,32.109091,43.409091,40.563636,62.609091,28.540909,46.963636
1,82.572973,84.072973,29.183784,89.337838,38.197297,58.040541,96.437838,96.532432,97.243243,91.783784,...,86.337838,99.602703,44.370270,51.478378,88.862162,95.270270,92.056757,81.372973,90.881081,97.927027
2,14.045833,16.391667,81.887500,2.825000,79.779167,37.625000,36.262500,44.133333,32.745833,72.683333,...,67.008333,43.862500,87.091667,48.016667,22.983333,26.154167,18.750000,49.204167,11.008333,13.225000
3,29.554545,56.406061,54.869697,83.848485,55.984848,59.442424,80.481818,90.693939,97.015152,73.148485,...,75.806061,96.578788,75.490909,63.330303,66.466667,61.863636,57.127273,73.696970,50.069697,62.409091
4,23.604545,23.018182,64.250000,7.927273,64.422727,40.163636,45.054545,71.363636,62.290909,86.768182,...,43.636364,72.931818,82.909091,37.368182,12.800000,33.881818,27.550000,73.213636,13.677273,26.995455


In [54]:
df_cluster - df_cluster.loc[0]

,Adol Birth Rate,Air Poll Mort,Alcohol Use,Cert Death Reg,Child Overweight,Child Sex Abuse,Child Stunting,Child Wasting,Conflict Mort,Disaster Mort,...,Sexual Violence,Skilled Birth Attend,Smoking Prev,Suicide Mort,TB Incid,UHC Index,Under-5 Mort,Vaccine Cov,WaSH Mort,Water
cluster,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,44.100246,61.309337,-48.870762,76.710565,-45.152703,15.795086,50.796929,40.836978,14.293243,9.415602,...,26.292383,23.743612,-5.143366,-1.985258,56.753071,51.861179,51.493120,18.763882,62.340172,50.963391
2,-24.426894,-6.371970,3.832955,-9.802273,-3.570833,-4.620455,-9.378409,-11.562121,-50.204167,-9.684848,...,6.962879,-31.996591,37.578030,-5.446970,-9.125758,-17.254924,-21.813636,-13.404924,-17.532576,-33.738636
3,-8.918182,33.642424,-23.184848,71.221212,-27.365152,17.196970,34.840909,34.998485,14.065152,-9.219697,...,15.760606,20.719697,25.977273,9.866667,34.357576,18.454545,16.563636,11.087879,21.528788,15.445455
4,-14.868182,0.254545,-13.804545,-4.700000,-18.927273,-2.081818,-0.586364,15.668182,-20.659091,4.400000,...,-16.409091,-2.927273,33.395455,-16.095455,-19.309091,-9.527273,-13.013636,10.604545,-14.863636,-19.968182
5,12.722010,26.646890,-30.122967,68.746411,-38.636842,15.562440,37.785407,27.230861,13.123684,7.194976,...,21.428230,22.498804,-16.332057,-10.171531,28.219856,25.790909,25.731100,7.775120,43.161722,38.886364
6,19.390431,23.820574,13.919139,47.209569,-46.760526,26.354545,33.259091,22.230861,-38.202632,10.968660,...,27.596651,20.088278,12.096890,32.320574,35.780383,27.348804,22.646890,18.827751,35.943301,31.315311


In [51]:
df_cluster = df_cluster.reset_index().melt(id_vars='cluster')
df_cluster.head()

,cluster,variable,value
0,0,Adol Birth Rate,38.472727
1,1,Adol Birth Rate,82.572973
2,2,Adol Birth Rate,14.045833
3,3,Adol Birth Rate,29.554545
4,4,Adol Birth Rate,23.604545


In [49]:
df_cluster.value.max()

99.60270270270269